In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [75]:
# 1.1

temp_data = pd.read_csv("GlobalLandTemperaturesByState.csv")
temp_data = temp_data.drop(['Country', 'AverageTemperatureUncertainty'], axis=1)

temp_data = temp_data.sort_values(by='dt', ascending = True)

temp_data = temp_data.where(temp_data['dt']>='2000')
temp_data = temp_data.dropna()

wyoming = temp_data[temp_data['State'].str.startswith('Wyoming')]
nebraska = temp_data[temp_data['State'].str.startswith('Nebraska')]
south_dakota = temp_data[temp_data['State'].str.startswith('South Dakota')]

sorted_temps = pd.concat([wyoming, nebraska, south_dakota])
sorted_temps

,dt,AverageTemperature,State
626275,2000-01-01,-4.290,Wyoming
626276,2000-02-01,-1.310,Wyoming
626277,2000-03-01,1.015,Wyoming
626278,2000-04-01,5.927,Wyoming
626279,2000-05-01,10.958,Wyoming
...,...,...,...
527354,2013-05-01,13.795,South Dakota
527355,2013-06-01,19.085,South Dakota
527356,2013-07-01,22.649,South Dakota
527357,2013-08-01,22.501,South Dakota


In [95]:
# 1.2

mean_temp = sorted_temps.groupby('dt')['AverageTemperature'].mean()

mean_temp
avg_temps = pd.DataFrame(mean_temp).reset_index()
avg_temps

,dt,AverageTemperature
0,2000-01-01,-4.072000
1,2000-02-01,-0.205000
2,2000-03-01,3.642667
3,2000-04-01,7.609667
4,2000-05-01,14.255000
...,...,...
160,2013-05-01,13.196000
161,2013-06-01,18.783667
162,2013-07-01,22.110333
163,2013-08-01,21.890000
